In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

In [ ]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup

In [37]:
movie_box = soup.find_all('div', class_ = "ipc-metadata-list-summary-item__c")

titles = []
numbers = []
ratings = []

for box in movie_box:
    #titles
    if box.find('h3', class_ = "ipc-title__text") is not None:
        title = box.find('h3', class_ = "ipc-title__text")
        titles.append(title.text)
    else:
        title.append('None')

    #numbers
    if box.find('ul', attrs={'data-testid': 'title-metadata-box-office-data-container'}) is not None:
        number = box.find('ul', attrs={'data-testid': 'title-metadata-box-office-data-container'})
        numbers.append(number.text)
    else:
        numbers.append('None')

    #ratings
    if box.find('span', attrs={'data-testid': 'ratingGroup--imdb-rating'}) is not None:
        rating = box.find('span', attrs={'data-testid': 'ratingGroup--imdb-rating'})
        ratings.append(rating.text.replace('\xa0', ''))
    else:
        ratings.append('None')

movie_df = pd.DataFrame({
    'title': titles,
    'number': numbers,
    'rating': ratings
})

movie_df

,title,number,rating
0,Sinners,Weekend Gross: $48MTotal Gross: $64MWeeks Rele...,8.2(36K)
1,Minecraft de film,Weekend Gross: $40MTotal Gross: $353MWeeks Rel...,5.9(70K)
2,The King of Kings,Weekend Gross: $18MTotal Gross: $49MWeeks Rele...,7.5(3.4K)
3,The Amateur,Weekend Gross: $7MTotal Gross: $29MWeeks Relea...,6.7(14K)
4,Warfare,Weekend Gross: $4.9MTotal Gross: $17MWeeks Rel...,7.9(12K)
5,Drop,Weekend Gross: $3.3MTotal Gross: $14MWeeks Rel...,6.5(7.3K)
6,Gekijô-ban Purojekuto Sekai Kowareta Sekai to ...,Weekend Gross: $2.8MTotal Gross: $2.8MWeeks Re...,6.4(721)
7,Pride & Prejudice,Weekend Gross: $2.8MTotal Gross: $42MWeeks Rel...,7.8(351K)
8,Episode #5.6,Weekend Gross: $1.5MTotal Gross: $3.8MWeeks Re...,7.8(151)
9,Snow White,Weekend Gross: $1.2MTotal Gross: $85MWeeks Rel...,1.6(345K)


In [28]:
movie_df.number[0]

'Weekend Gross: $48MTotal Gross: $64MWeeks Released: 1'

In [38]:
movie_df['number'] = movie_df['number'].str.replace('Weekend Gross: ','').str.replace('Total Gross: ','|').str.replace('Weeks Released: ','|')

movie_df[['weekend gross', 'total gross', 'weeks released']] = movie_df['number'].str.split('|', expand = True)
movie_df[['ratings', 'vote_count']] = movie_df['rating'].str.split('(', expand = True)
movie_df.drop(['number', 'rating'], axis=1, inplace=True)
movie_df['vote_count'] = movie_df['vote_count'].str.replace(')', '')
movie_df.to_csv('movie_data.csv', index=False)
